In [ ]:
#include <ql/math/integrals/exponentialintegrals.hpp>
#include <ql/math/integrals/gausslobattointegral.hpp>
#include <ql/math/integrals/kronrodintegral.hpp>
#include <ql/math/integrals/simpsonintegral.hpp>
#include <ql/math/integrals/trapezoidintegral.hpp>
#include <ql/math/integrals/expsinhintegral.hpp>
#include <ql/math/solvers1d/brent.hpp>

from enum import Enum
from scipy import integrate
import numpy as np

In [ ]:
I1 = integrate.simpson(y1, x=x)
integrate.trapezoid(), integrate

In [ ]:
class integrand1:
    def __init__(self, c_inf, f):
        self.c_inf = c_inf
        self.f = f

    def __call__(self, x):
        return np.where(
            (1.0-x)*self.c_inf > QL_EPSILON, 
            self.f(-np.log(0.5-0.5*x)/self.c_inf)/((1.0-x)*self.c_inf), 
            0.0
        )
        #if ((1.0-x)*self.c_inf > QL_EPSILON):
        #    return self.f(-np.log(0.5-0.5*x)/self.c_inf)/((1.0-x)*self.c_inf)
        #else:
        #    return 0.0

class integrand2:
    def __init__(self, c_inf, f): 
        self.c_inf = c_inf
        self.f = f
    
    def __call__(self, x):
        return np.where((x*self.c_inf > QL_EPSILON), self.f(-np.log(x)/self.c_inf)/(x*self.c_inf), 0.0)
        #if (x*self.c_inf > QL_EPSILON):
        #     return self.f(-np.log(x)/self.c_inf)/(x*self.c_inf)
        #else:
        #    return 0.0
        
class integrand3:
    def __init__(self, c_inf, f) -> None:
        self.inte2 = integrand2(c_inf, f)
     
    def __call__(self, x):
        return self.inte2(1.0-x)


In [ ]:
class Fj_Helper:
    def __init__(
            self, 
            kappa, 
            theta, 
            sigma, 
            v0,
            s0, 
            rho,  
            engine: AnalyticHestonEngine,
            cpx_log: ComplexLogFormula,
    term: Time, 
    strike, 
    ratio, 
    j: int
):
        self.j= j
        #   VanillaOption::arguments& arg_;
        Real kappa
        theta_
        sigma_ 
        v0_
        cpx_log: ComplexLogFormula, 
        
        # helper variables
        self.term_
        x_ 
        sx_
        dd_
        sigma2_ 
        rsigma_
        t0

        # log branch counter
        mutable int  b_ = 0;     # log branch counter
        mutable Real g_km1_ = 0; # imag part of last log value

        const AnalyticHestonEngine* constengine

    AnalyticHestonEngine::Fj_Helper::Fj_Helper(Real kappa, Real theta,
        Real sigma, Real v0, Real s0, Real rho,
        const AnalyticHestonEngine* const engine,
        ComplexLogFormula cpxLog,
        Time term, Real strike, Real ratio, Size j)
        :
        j_(j),
        kappa_(kappa),
        theta_(theta),
        sigma_(sigma),
        v0_(v0),
        cpxLog_(cpxLog),
        term_(term),
        x_(std::log(s0)),
        sx_(std::log(strike)),
        dd_(x_-std::log(ratio)),
        sigma2_(sigma_*sigma_),
        rsigma_(rho*sigma_),
        t0_(kappa - ((j== 1)? rho*sigma : Real(0))),
        
        engine_(engine)
    {
    }

    Real AnalyticHestonEngine::Fj_Helper::operator()(Real phi) const
    {
        const Real rpsig(rsigma_*phi);

        const std::complex<Real> t1 = t0_+std::complex<Real>(0, -rpsig);
        const std::complex<Real> d =
            std::sqrt(t1*t1 - sigma2_*phi
                      *std::complex<Real>(-phi, (j_== 1)? 1 : -1));
        const std::complex<Real> ex = std::exp(-d*term_);
        const std::complex<Real> addOnTerm =
            engine_ != nullptr ? engine_->addOnTerm(phi, term_, j_) : Real(0.0);

        if (cpxLog_ == Gatheral) {
            if (phi != 0.0) {
                if (sigma_ > 1e-5) {
                    const std::complex<Real> p = (t1-d)/(t1+d);
                    const std::complex<Real> g
                                            = std::log((1.0 - p*ex)/(1.0 - p));

                    return
                        std::exp(v0_*(t1-d)*(1.0-ex)/(sigma2_*(1.0-ex*p))
                                 + (kappa_*theta_)/sigma2_*((t1-d)*term_-2.0*g)
                                 + std::complex<Real>(0.0, phi*(dd_-sx_))
                                 + addOnTerm
                                 ).imag()/phi;
                }
                else {
                    const std::complex<Real> td = phi/(2.0*t1)
                                   *std::complex<Real>(-phi, (j_== 1)? 1 : -1);
                    const std::complex<Real> p = td*sigma2_/(t1+d);
                    const std::complex<Real> g = p*(1.0-ex);

                    return
                        std::exp(v0_*td*(1.0-ex)/(1.0-p*ex)
                                 + (kappa_*theta_)*(td*term_-2.0*g/sigma2_)
                                 + std::complex<Real>(0.0, phi*(dd_-sx_))
                                 + addOnTerm
                                 ).imag()/phi;
                }
            }
            else {
                // use l'Hospital's rule to get lim_{phi->0}
                if (j_ == 1) {
                    const Real kmr = rsigma_-kappa_;
                    if (std::fabs(kmr) > 1e-7) {
                        return dd_-sx_
                            + (std::exp(kmr*term_)*kappa_*theta_
                               -kappa_*theta_*(kmr*term_+1.0) ) / (2*kmr*kmr)
                            - v0_*(1.0-std::exp(kmr*term_)) / (2.0*kmr);
                    }
                    else
                        # \kappa = \rho * \sigma
                        return dd_-sx_ + 0.25*kappa_*theta_*term_*term_
                                       + 0.5*v0_*term_;
                }
                else {
                    return dd_-sx_
                        - (std::exp(-kappa_*term_)*kappa_*theta_
                           +kappa_*theta_*(kappa_*term_-1.0))/(2*kappa_*kappa_)
                        - v0_*(1.0-std::exp(-kappa_*term_))/(2*kappa_);
                }
            }
        }
        else if (cpxLog_ == BranchCorrection) {
            const std::complex<Real> p = (t1+d)/(t1-d);

            // next term: g = std::log((1.0 - p*std::exp(d*term_))/(1.0 - p))
            std::complex<Real> g;

            // the exp of the following expression is needed.
            const std::complex<Real> e = std::log(p)+d*term_;

            // does it fit to the machine precision?
            if (std::exp(-e.real()) > QL_EPSILON) {
                g = std::log((1.0 - p/ex)/(1.0 - p));
            } else {
                // use a "big phi" approximation
                g = d*term_ + std::log(p/(p - 1.0));

                if (g.imag() > np.pi or g.imag() <= -M_PI) {
                    // get back to principal branch of the complex logarithm
                    Real im = std::fmod(g.imag(), 2*M_PI);
                    if (im > M_PI)
                        im -= 2*M_PI;
                    else if (im <= -M_PI)
                        im += 2*M_PI;

                    g = std::complex<Real>(g.real(), im);
                }
            }

            // be careful here as we have to use a log branch correction
            // to deal with the discontinuities of the complex logarithm.
            // the principal branch is not always the correct one.
            // (s. A. Sepp, chapter 4)
            // remark: there is still the change that we miss a branch
            // if the order of the integration is not high enough.
            const Real tmp = g.imag() - g_km1_;
            if (tmp <= -M_PI)
                ++b_;
            else if (tmp > M_PI)
                --b_;

            g_km1_ = g.imag();
            g += std::complex<Real>(0, 2*b_*M_PI);

            return std::exp(v0_*(t1+d)*(ex-1.0)/(sigma2_*(ex-p))
                            + (kappa_*theta_)/sigma2_*((t1+d)*term_-2.0*g)
                            + std::complex<Real>(0,phi*(dd_-sx_))
                            + addOnTerm
                            ).imag()/phi;
        }
        else {
            QL_FAIL("unknown complex logarithm formula");
        }
    }


In [ ]:
def heston_pv(
        riskFreeDiscount, 
        dividendDiscount, 
        spotPrice,
        strikePrice,
        term,
        kappa, 
        theta,
        sigma,
        v0, 
        rho,
        TypePayoff& type,
        Integration& integration,
        cpx_log: ComplexLogFormula,
        engine: AnalyticHestonEngine,
        value,
        evaluations
):

        ratio = riskFreeDiscount/dividendDiscount

        evaluations = 0

        match cpx_log:
          case Gatheral:
          case BranchCorrection:
            c_inf = np.min(0.2, np.max(0.0001, np.sqrt(1.0-rho*rho)/sigma))*(v0 + kappa*theta*term)

            p1 = integration.calculate(c_inf,
                                       Fj_Helper(kappa, theta, sigma, v0, spotPrice, rho, enginePtr, cpxLog, term, strikePrice, ratio, 1))/np.pi
            
            evaluations += integration.numberOfEvaluations()

            p2 = integration.calculate(c_inf, Fj_Helper(kappa, theta, sigma, v0, spotPrice, rho, engine, cpx_log, term, strikePrice, ratio, 2))/np.pi
            
            evaluations += integration.numberOfEvaluations();

            match option_type:
              case Option.Call:
                value = spotPrice*dividendDiscount*(p1+0.5) - strikePrice*riskFreeDiscount*(p2+0.5)
                break
              case Option.Put:
                value = spotPrice*dividendDiscount*(p1-0.5) - strikePrice*riskFreeDiscount*(p2-0.5)
                break
              default:
                raise ValueError("unknown option type")
            }
          }
          case AndersenPiterbarg:
          case AndersenPiterbargOptCV:
          case AsymptoticChF:
          case OptimalCV:
            c_inf = np.sqrt(1.0-rho*rho)*(v0 + kappa*theta*term)/sigma

            fwdPrice = spotPrice / ratio

            epsilon = engine.andersenPiterbargEpsilon_*np.pi/(np.sqrt(strikePrice*fwdPrice)*riskFreeDiscount)

            const ext::function<Real()> uM = [&](){
                return Integration::andersenPiterbargIntegrationLimit(c_inf, epsilon, v0, term);
            };

            AP_Helper cvHelper(term, fwdPrice, strikePrice,
                (cpxLog == OptimalCV)
                    ? optimalControlVariate(term, v0, kappa, theta, sigma, rho)
                    : cpxLog,
                enginePtr
            );

            cvValue = cvHelper.controlVariateValue()

            const Real h_cv = integration.calculate(c_inf, cvHelper, uM) * np.sqrt(strikePrice * fwdPrice)/np.pi
            
            evaluations += integration.numberOfEvaluations()

            switch (type.optionType())
            {
              case Option.Call:
                value = (cvValue + h_cv)*riskFreeDiscount;
                break
              case Option::Put:
                value = (cvValue + h_cv - (fwdPrice - strikePrice))*riskFreeDiscount;
                break
              default:
                raise ValueError("unknown option type")
          break

          default:
            QL_FAIL("unknown complex log formula");
        }
    }
}